In [1]:
#from kedro.extras.datasets.pandas import CSVDataSet
#import pandas as pd 
%reload_kedro

[05/09/23 21:28:30] INFO     Resolved project path as: H:\kedro_1\weather_us\weather-us.            ]8;id=630941;file://H:\kedro_1\weather_us\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=541591;file://H:\kedro_1\weather_us\venv\lib\site-packages\kedro\ipython\__init__.py#134\134]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[05/09/23 21:28:33] INFO     Kedro project weather-us                                               ]8;id=814484;file://H:\kedro_1\weather_us\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=360619;file://H:\kedro_1\weather_us\venv\lib\site-packages\kedro\ipython\__init__.py#103\103]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=725472;file://H:\kedro_1\weather_us\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=617421;file://H:\kedro_1\weather_us\venv\lib\site-packages\kedro\ipython\__init__.py#104\104]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=761027;file://H:\kedro_1\weather_us\venv\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=988324;file://H:\kedro_1\weather_us\venv\lib\site-packages\kedro\ipython\__init__.py#110\110]8;;\

In [2]:
catalog.list()

['weather_aus_raw',
 'df1',
 'df1_treat_training_data',
 'X_training',
 'y_training',
 'df2',
 'logreg',
 'parameters']

In [3]:
import numpy as np  #linear algebra
import pandas as pd #data processing, CSV file I?O
import matplotlib.pyplot as plt #data visualization 
import seaborn as sns #statistical visualization 
%matplotlib inline 
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression

In [4]:
df = catalog.load("weather_aus_raw")

[05/09/23 21:28:50] INFO     Loading data from 'weather_aus_raw' (CSVDataSet)...                ]8;id=502487;file://H:\kedro_1\weather_us\venv\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=54050;file://H:\kedro_1\weather_us\venv\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

In [5]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [6]:
df.shape

(142193, 24)

In [7]:
df.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RISK_MM', 'RainTomorrow'],
      dtype='object')

In [8]:
df.nunique().sum()

8703

In [9]:
# dropping the null values in dependant variable
# use df1 variable for training data

def extract_training_data(df):
  """
  Drop date column from dataframe.
  Arg=df
  output=Dataframe with no date column.
  """
  df=df.drop(["Date"],axis=1)
  df1=df[df['RainTomorrow'].notna()] 
  return df1

**Dataset Reading Or Dataset Basic information**

In [10]:
df1=extract_training_data(df)

In [11]:
df1.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [12]:
df1.shape

(142193, 23)

In [13]:
df1.columns

Index(['Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RISK_MM', 'RainTomorrow'],
      dtype='object')

In [14]:
df1.size

3270439

In [15]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       142193 non-null  object 
 1   MinTemp        141556 non-null  float64
 2   MaxTemp        141871 non-null  float64
 3   Rainfall       140787 non-null  float64
 4   Evaporation    81350 non-null   float64
 5   Sunshine       74377 non-null   float64
 6   WindGustDir    132863 non-null  object 
 7   WindGustSpeed  132923 non-null  float64
 8   WindDir9am     132180 non-null  object 
 9   WindDir3pm     138415 non-null  object 
 10  WindSpeed9am   140845 non-null  float64
 11  WindSpeed3pm   139563 non-null  float64
 12  Humidity9am    140419 non-null  float64
 13  Humidity3pm    138583 non-null  float64
 14  Pressure9am    128179 non-null  float64
 15  Pressure3pm    128212 non-null  float64
 16  Cloud9am       88536 non-null   float64
 17  Cloud3pm       85099 non-null

**Data Preprocessing OR Feature Engineering**

Filling Missing value for catgorical column

In [16]:
# Get list of categorical variables
s = (df1.dtypes == "object")
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']


In [17]:
# Get a list of numerical variable

s1=(df1.dtypes=='float64')
object_cols2=list(s1[s1].index)

print('Numerical Variable : \n{}'.format(object_cols2))

Numerical Variable : 
['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm', 'RISK_MM']


In [18]:
# Define a function to fill missing values with the mode of a categorical column
'''
def fillna_catgorical_mode(column):
    """
    Filling Missing value in catgorical column
    """
    column_mode = column.mode()[0] # mode() returns a Series, so we take the first value
    
    return column.fillna(column_mode)


    # Apply the function to each categorical column in the DataFrame
categorical_cols = df1.select_dtypes(include=['object']).columns
df1[categorical_cols] = df1[categorical_cols].apply(fillna_catgorical_mode)
'''
#print(df1)

'\ndef fillna_catgorical_mode(column):\n    """\n    Filling Missing value in catgorical column\n    """\n    column_mode = column.mode()[0] # mode() returns a Series, so we take the first value\n    \n    return column.fillna(column_mode)\n\n\n    # Apply the function to each categorical column in the DataFrame\ncategorical_cols = df1.select_dtypes(include=[\'object\']).columns\ndf1[categorical_cols] = df1[categorical_cols].apply(fillna_catgorical_mode)\n'

In [19]:
def treat_missing(df1):
  """
  treat missing values with ffill and bfill method

  Arg=df_training_data

  output=nan values from training data get fill with the ffill and bfill method."""
  df1_treat_training_data=df1.fillna(method="ffill",axis=0).fillna(method="bfill",axis=0)
  return df1_treat_training_data

In [20]:
def training_data_split(df1_treat_training_data):
  """
  Training data spliting ie.separating predictors and response variables.

  Arg=df_treat_training_data

  Output=Target variable get separated from main traing data."""
  X_training=df1_treat_training_data.drop(["RainTomorrow"],axis=1)
  y_training=df1_treat_training_data["RainTomorrow"]
  return X_training, y_training

In [21]:
def lebel_encoding_filling_null(X_training):
  """
  Labal encoding on the discrite varibales.

  Arg=X_training

  Output=converting categorical data into numeric variable."""
  
  label_encoder = preprocessing.LabelEncoder()
  
  X_training["Location"]=label_encoder.fit_transform(X_training["Location"])
  X_training["WindGustDir"]=label_encoder.fit_transform(X_training["WindGustDir"])
  X_training["WindDir9am"]=label_encoder.fit_transform(X_training["WindDir9am"])
  X_training["WindDir3pm"]=label_encoder.fit_transform(X_training["WindDir3pm"])
  X_training["RainToday"]=label_encoder.fit_transform(X_training["RainToday"])
  df2 = X_training.copy()
  return df2

In [22]:
def split_data(df2,y_training):
  """Split data into features and targets training and test set.
  Args:
    data;data containing features and targets
  returns:
    split data
 """
  

  X_train,X_test,y_train,y_test=train_test_split(df2,y_training,random_state=0,test_size=0.20)

  return X_train,X_test,y_train,y_test 

In [23]:
def train_model(X_train,X_test,y_train,y_test ):
  """Trains the logistic regression model.
  Args:
      X_train:Training data of independent features
      y_train:Training data for price
  Returns:
      Trined model.
 """
  #instantiate the model
  logreg=LogisticRegression(solver='liblinear',random_state=0)

  #fit the model
  logreg = logreg.fit(X_train,y_train)
  return logreg


In [24]:
def evaluate_model(logreg:LogisticRegression, X_test:pd.DataFrame, y_test:pd.Series):
  """calculate and logs the coefficient of determination.
  Args:
      logreg:Trined model
      X_test:Testing data of independent features
      y_test:Testing data for price
  Returns:
      y_pred_test:prediction on x test
      acc        :accuracy_score
  """
  y_pred_test=logreg.predict(X_test)
  #Check accuracy score
  
  acc=(accuracy_score(y_test,y_pred_test))
  print('model accuracy_score: ',acc)
  return y_pred_test,acc

In [25]:
def inferance_data(df):
  """Drop date column from dataframe.
  Arg=df
  output=Dataframe with no date column."""
  df=df.drop(["Date"],axis=1)
  df_inferance=df[df['RainTomorrow'].isnana()] 
  return df_inferance

In [26]:
def treat_missing_inferance(df_inferance):
  """treat missing values with ffill and bfill method

  Arg=df_training_data

  output=nan values from training data get fill with the ffill and bfill method."""
  df1_treat_inferance_data=df_inferance.fillna(method="ffill",axis=0).fillna(method="bfill",axis=0)
  return df1_treat_inferance_data

In [27]:
def inferance_data_split(df1_treat_inferance_data):
  """Training data spliting ie.separating predictors and response variables.

  Arg=df_treat_training_data

  Output=Target variable get separated from main traing data."""
  X_inferance=df1_treat_inferance_data.drop(["RainTomorrow"],axis=1)
  y_inferance=df1_treat_inferance_data["RainTomorrow"]
  return X_inferance, y_inferance

In [28]:
def lebel_encoding_filling_null(X_inferance):
  """Lebal encoding on the discrite varibales.

  Arg=X_training

  Output=converting categorical data into numeric variable."""
  
  label_encoder = preprocessing.LabelEncoder()
  
  X_inferance["Location"]=label_encoder.fit_transform(X_inferance["Location"])
  X_inferance["WindGustDir"]=label_encoder.fit_transform(X_inferance["WindGustDir"])
  X_inferance["WindDir9am"]=label_encoder.fit_transform(X_inferance["WindDir9am"])
  X_inferance["WindDir3pm"]=label_encoder.fit_transform(X_inferance["WindDir3pm"])
  X_inferance["RainToday"]=label_encoder.fit_transform(X_inferance["RainToday"])
  df2_inferance = X_inferance.copy()
  return df2_inferance